Để 1 feature trở nên hữu dụng, nó phải liên hệ với cấu hình model.  
VD : Linear Model cần feature có quan hệ tuyến tính với nhau  

Vì thế, kĩ thuật FE bao gồm việc biến đổi data thành feature phù hợp với mô hình. Như length và width không cung cấp quá nhiều ý nghĩa, nhưng Area = L.W lại phù hợp với mô hình giá nhà

Trước hết, ta cần tạo 1 baseline (mô hình gốc, chưa FE, data preprocessing) để làm điểm tham chiếu sau này

In [167]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

In [168]:
data = pd.read_csv('Concrete_Data_Yeh.csv')
data.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [169]:
y = data['csMPa']
X = data.drop(columns= 'csMPa')

hàm tính mae

In [170]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
def mae_cal(Xi, yi) :
    X_train, X_test, y_train, y_test = train_test_split(Xi,yi)
    X_train2, X_label, y_train2, y_label = train_test_split(X_train, y_train)
    XGBR = XGBRegressor(early_stopping_rounds = 5, random_state = 1, n_estimators = 800, learning_rate = 0.1, eval_metric = 'mae')
    XGBR.fit(X_train2, y_train2,
          eval_set = [(X_label, y_label)], verbose = 0)
    return mean_absolute_error(XGBR.predict(X_test), y_test)

Bắt đầu FE, chọn feature, lập tỉ lệ (trộn xi măng cần tỉ lệ hơn là giá trị tuyệt đối)


In [171]:
data.corr(numeric_only=True).abs()[['csMPa']]

,csMPa
cement,0.497832
slag,0.134829
flyash,0.105755
water,0.289633
superplasticizer,0.366079
coarseaggregate,0.164935
fineaggregate,0.167241
age,0.328873
csMPa,1.000000


In [172]:
baseline_mae = mae_cal(X,y)

In [173]:
featureX = X.copy()

In [174]:
featureX['FCRatio'] = X['coarseaggregate']/X['fineaggregate']
featureX['CWRatio'] = X['cement']/X['water']
featureX['FFCRatio'] = (X["coarseaggregate"] + X["fineaggregate"]) / X["cement"]

In [175]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
RFR = RandomForestRegressor(random_state = 1) 
X_train, X_test, y_train, y_test = train_test_split(X,y)
RFR.fit(X_train, y_train)
RFR_mae = mean_absolute_error(RFR.predict(X_test), y_test)

In [176]:
featured_mae = mae_cal(featureX,y)
print(f"featured_mae : {featured_mae}, base mae : {baseline_mae}, RFR mae :  {RFR_mae}")

featured_mae : 3.1864316897429235, base mae : 3.468476137967073, RFR mae :  3.453071808785529
